In [11]:
import math
import pandas as pd


import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [12]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [13]:
df = pd.read_csv('data/sorted_train.csv')

In [22]:
n_classes = 4

counts = {'A': 325198, 'C': 183718, 'D': 27039, 'B': 15371}
total_count = sum(counts.values())

weightsArray =[]
for i, _class in enumerate(['A', 'B', 'C', 'D']):
    weightsArray.append(math.log(total_count/max(counts[_class],1))+1)

# approach from stack overflow, 
# https://datascience.stackexchange.com/questions/12886/tensorflow-adjusting-cost-function-for-imbalanced-data?newreg=1ec2de91ae154cd0ad5c459b3a2adfaf
print (weightsArray)

# or just actual weights... uhmmm.
print({i: counts[x]/total_count for i, x in enumerate(['A', 'B', 'C', 'D'])})

# Whwoops, first go I had created weights which were proportional 
# instead of inversely proportional.
# [1.5278920581455526, 2.098924312471526, 4.015046017496413, 4.579843668496]
# {0: 0.5898470233582309, 1: 0.027880056445732652, 2: 0.33322934162364914, 3: 0.0490435785723873}

[1.5278920581455526, 4.579843668496, 2.098924312471526, 4.015046017496413]
{0: 0.5898470233582309, 1: 0.027880056445732652, 2: 0.33322934162364914, 3: 0.0490435785723873}


In [ ]:
# # https://datascience.stackexchange.com/questions/12886/tensorflow-adjusting-cost-function-for-imbalanced-data?newreg=1ec2de91ae154cd0ad5c459b3a2adfaf
# 
class_weight = tf.constant(weightsArray)
weighted_logits = tf.mul(pred, class_weight)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(weighted_logits, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# And that question's answer,...
# Take the cost like normal
error = tf.nn.softmax_cross_entropy_with_logits(pred, y)

# Scale the cost by the class weights
scaled_error = tf.mul(error, class_weight)

# Reduce
cost = tf.reduce_mean(scaled_error)

In [29]:
# Construct a `Session` to execute the graph.
# sess = tf.Session()
# Execute the graph and store the value that `e` represents in `result`.
# result = sess.run(tensor)

# init_op = tf.global_variables_initializer()   # ?
class_weights_d = {0: 0.5898470233582309,
                     1: 0.027880056445732652,
                     2: 0.33322934162364914,
                     3: 0.0490435785723873}
class_weights_v = tf.constant(np.array([class_weights_d[k] for k in range(3)]))    
#class_weights_v = [1.5278920581455526, 4.579843668496, 2.098924312471526, 4.015046017496413]
class_weights_v = [[1],[1],[1]]

with tf.Session() as sess:
    target = np.array([[1.,0.,0.,0.], # target.
                       [0.,1.,0.,0.],
                       [0.,0.,1.,0.],
                      ])
    prediction = np.array([[1.,0.,0.,0.], # prediction
                           [1.,0.,0.,0.],
                           [1.,0.,0.,0.],
                           ])
    error = tf.nn.softmax_cross_entropy_with_logits(logits=prediction, 
                                                    labels=target)

    # (labels=target,  logits=output)
    
    scaled_error = tf.multiply(error, class_weights_v)
    result = sess.run(scaled_error)    
    # acc, l, soft_max_a = sess.run([accuracy, loss, a], feed_dict={x: X_t, y: Y_t})
    
#     tf.add
print(result)    

[[0.74366838 1.74366838 1.74366838]
 [0.74366838 1.74366838 1.74366838]
 [0.74366838 1.74366838 1.74366838]]


In [21]:
result

array([0.43865058, 0.04861357, 0.58104147])

In [ ]:
def bake_model4_may18(x_train, y_train, x_test, y_test, epochs=1, **kwargs):
    
    model = Sequential()
    model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2,
                   input_shape=(None, 1)))

    # 4 because 'A', 'B', 'C', 'D'.
    model.add(Dense(4))

    model.compile(loss='categorical_crossentropy', # unbalanced_categorical_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
    
    # 🤔 model.compile(loss='categorical_crossentropy', optimizer='adam')
    # initial_state ... for the LSTM , hmm

    monitor = EarlyStopping(monitor='val_loss',
                            min_delta=1e-3, patience=5, verbose=1, mode='auto')
    print('Train...', timestamp())

    # https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#arguments_8
    # - hmm so fit() can take a generator sometimes.
    # - use_multiprocessing=True 
    model.fit(x_train, y_train, validation_data=(x_test, y_test),
              callbacks=[monitor], verbose=2, epochs=epochs)

    return model

# forking from here:
# https://github.com/keras-team/keras/blob/master/keras/backend/tensorflow_backend.py#L3527
def unbalanced_categorical_crossentropy(target, output, from_logits=False, axis=-1):
    """Categorical crossentropy between an output tensor and a target tensor.
    # Arguments
        target: A tensor of the same shape as `output`.
        output: A tensor resulting from a softmax
            (unless `from_logits` is True, in which
            case `output` is expected to be the logits).
        from_logits: Boolean, whether `output` is the
            result of a softmax, or is a tensor of logits.
        axis: Int specifying the channels axis. `axis=-1`
            corresponds to data format `channels_last`,
            and `axis=1` corresponds to data format
            `channels_first`.
    # Returns
        Output tensor.
    # Raises
        ValueError: if `axis` is neither -1 nor one of
            the axes of `output`.
    """
    output_dimensions = list(range(len(output.get_shape())))
    if axis != -1 and axis not in output_dimensions:
        raise ValueError(
            '{}{}{}'.format(
                'Unexpected channels axis {}. '.format(axis),
                'Expected to be -1 or one of the axes of `output`, ',
                'which has {} dimensions.'.format(len(output.get_shape()))))
        
    # Pre-computed Hard-coded class_weights...
    class_weights = {0: 0.5898470233582309,
                     1: 0.027880056445732652,
                     2: 0.33322934162364914,
                     3: 0.0490435785723873}
    # Note: tf.nn.softmax_cross_entropy_with_logits
    # expects logits, Keras expects probabilities.
    if not from_logits:
        # scale preds so that the class probas of each sample sum to 1
        output /= tf.reduce_sum(output, axis, True)
        # manual computation of crossentropy
        _epsilon = _to_tensor(epsilon(), output.dtype.base_dtype)
        output = tf.clip_by_value(output, _epsilon, 1. - _epsilon)
        return - tf.reduce_sum(target * tf.log(output), axis)
    else:
        error = tf.nn.softmax_cross_entropy_with_logits(labels=target,
                                                       logits=output)
    
        # Scale by class
        class_weight = 
        return  tf.mul(error, class_weight)



In [31]:
# ok ran model-4 on Sagemaker notebook w/ one change, making use of the categorical 
restored_model4 = keras.models.load_model('models/2019-05-19T001217-UTC-model-4.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [38]:
import datetime
import itertools
from collections import Counter

In [39]:
# do preds... get confusion..
outdata = make_data(df, crews={'training': [1],
                        'test': [2]},
              sequence_window=256, percent_of_data=1,
             feature_cols={'r': simple_scaler})

Start building training set 2019-05-19 00:08:20 EST
Start building testing set 2019-05-19 00:08:26 EST


In [48]:
# 

%time preds, preds_classes = do_get_preds(outdata, restored_model4)

CPU times: user 56min 23s, sys: 21min 7s, total: 1h 17min 30s
Wall time: 21min 10s


In [49]:
confusion_yea = do_confusiony(outdata, preds_classes)

Instructions for updating:
Use tf.cast instead.


In [50]:
confusion_yea

array([[325198,      0,      0,      0],
       [ 15371,      0,      0,      0],
       [183718,      0,      0,      0],
       [ 27039,      0,      0,      0]], dtype=int32)

In [30]:
# Utils
encode_class = np.vectorize(lambda x: {'A': 0,
                                      'B': 1,
                                      'C': 2,
                                      'D': 3}.get(x))

decode_class = np.vectorize(lambda x: {0: 'A',
                                      1: 'B',
                                      2: 'C',
                                      3: 'D'}.get(x))

simple_scaler = lambda x, a: x*a 

def timestamp():
    return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S EST')

def do_standard_scaling(df, cols, scalar_dict=None):
    if scalar_dict is None:    
        scalar_dict = {col: StandardScaler().fit(df[[col]]) for col in cols}
        
    for col in cols:
        df[col + '_scaled'] = np.resize(
            scalar_dict[col].transform(df[[col]]),
            (df.shape[0],))
    
    return scalar_dict, df


def make_data(df, crews={'training': [1],
                        'test': [2]},
              sequence_window=256, percent_of_data=100,
             feature_cols={'r': 'standard_scaler'}):

    # current sorted as ['crew', 'experiment', 'time']
    [0, 1] # each seat
    ['CA', 'DA', 'SS'] # experiment
    
    sort_cols = ['crew', 'seat', 'experiment', 'time']
    target_col = 'event'
    
    what_cols = sort_cols + list(feature_cols) + [target_col]

    # Training
    traindf = df[df.crew.isin(crews['training'])][what_cols].sort_values(
        by=sort_cols).copy()
    
    scalar_dict, _ = do_standard_scaling(traindf, ['r'])
    
    print('Start building training set', timestamp())
    x_train, y_train = get_windows(traindf, ['r_scaled', 'event'],
                                   sequence_window,
                                  percent_of_data=percent_of_data)
    
    # Testing
    testdf = df[df.crew.isin(crews['test'])][what_cols].sort_values(
        by=sort_cols).copy()

    _, _ = do_standard_scaling(testdf, ['r'], scalar_dict)
    
    
    print('Start building testing set', timestamp())
    x_test, y_test = get_windows(testdf, ['r_scaled', 'event'],
                                 sequence_window,
                                 percent_of_data=percent_of_data)


    outdata = {
        "x_train": x_train,
        "y_train": reshape_y(encode_class(y_train), 4), # y_train,
        "x_test": x_test,
        "y_test": reshape_y(encode_class(y_test), 4), # y_test
        "y_train_original": y_train,
        "y_test_original": y_test,
        "traindf": traindf,
        "testdf": testdf,}
    metadata = {
        "metadata": {
            "output": {
                "shapes": {k: outdata[k].shape for k in list(outdata)},
                "Counter(outdata['y_train_original'])":
                dict(Counter(y_train)),
                "Counter(outdata['y_test_original'])":
                dict(Counter(y_test)),},
            "input": {"kwargs": {
                "crews": crews,
                "percent_of_data": percent_of_data,
                "sequence_window": sequence_window,
                "feature_cols": list(feature_cols)}},
            "data_ts": timestamp()
        }}
            
    return {**outdata, **metadata}
    
    
def runner(df):
    print('Start make_data', timestamp())
    outdata = make_data(df, crews={'training': [1],
                        'test': [2]},
              sequence_window=256, percent_of_data=1,
             feature_cols={'r': simple_scaler})
    
    validate_data(outdata)

    print('Start bake_model', timestamp())
    model = bake_model18(**outdata, epochs=2)
    return outdata, model

def validate_data(data):
    assert len(Counter(data['y_train_original'])) > 1
    assert len(Counter(data['y_test_original'])) > 1
  
    
def get_windows(df, cols, window_size, percent_of_data=100):
    
    whats_proportion_index = lambda x, y: round(x*y)
    
    X = []
    Y = []
    choices = (df.crew.unique().tolist(), [0, 1], ['CA', 'DA', 'SS'])
    for crew, seat, experiment in itertools.product(*choices):
        query = (df.crew == crew)&(df.seat == seat)&(df.experiment == experiment)
        thisdf = df[query][cols]
        X_i, Y_i = to_sequences(thisdf.values, window_size)
        X.append(X_i[:
                     whats_proportion_index(
                         X_i.shape[0],
                         percent_of_data)])
        Y.append(Y_i[:
                     whats_proportion_index(
                        Y_i.shape[0],
                        percent_of_data)])
        
    return np.concatenate(X), np.concatenate(Y)

# Borrowing parts of this func from 
# https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class10_lstm.ipynb
def to_sequences(obs, seq_size, incols=[0], outcols=[1]):
    x = []
    y = []

    for i in range(len(obs)-seq_size-1):
        #print(i)
        window = obs[i:(i+seq_size)][..., 0]
        after_window = obs[i+seq_size, 1] # FIXME :off by 1 error here?
        # window = [[x] for x in window]

        x.append(window)
        y.append(after_window)
        
    xarr = np.array(x)
    yarr = np.array(y)
    return (np.resize(xarr, xarr.shape + (1,)),
            yarr)
    
    

def reshape_y(y, num_cols):

    # y = np.array([1,2,3,2,3,1],dtype=np.int32)

    # Convert y2 to dummy variables
    y2 = np.zeros((y.shape[0], num_cols), dtype=np.float32)
    y2[np.arange(y.shape[0]), y] = 1.0
    return y2


In [47]:
def do_get_preds(outdata, model):
    preds = model.predict(outdata['x_test'])
    preds_classes = np.argmax(preds, axis=1)
    # print('Counter predict classes', Counter(preds_classes))
    # print('original', Counter(outdata['y_test_original']))
    # 
    return preds, preds_classes
    # 
    # 
    # preds, preds_classes = do_get_preds_and_confusion(outdata, model)
    
def do_confusiony(outdata, preds_classes):
    tensor = tf.confusion_matrix( 
    encode_class(outdata['y_test_original']),# labels
    preds_classes, # predictions
    num_classes=4)

    # Construct a `Session` to execute the graph.
    with tf.Session() as sess:

        # Execute the graph and store the value that `e` represents in `result`.
        result = sess.run(tensor)
        # print(result)
        return result
    

In [32]:
def make_s3_resource():
    s3 = boto3.resource('s3',
            # aws_access_key_id=os.getenv('S3_BLOG_UPLOAD_ACCESS_KEY'),
            # aws_secret_access_key=os.getenv('S3_BLOG_UPLOAD_SECRET'),
            region_name='us-east-1')
    return s3


def write_s3_file(bucket_name, s3_filename, content):
    # assert isinstance(content, basestring)
    s3conn = make_s3_resource()
    # try:
    s3conn.Object(bucket_name, s3_filename).put(
            Body=content)
    # return True
    # except botocore.exceptions.ClientError as e:
    # return False


def read_s3_file(bucket_name, s3_filename):
    s3conn = make_s3_resource()
    # try:
    return s3conn.Object(bucket_name, s3_filename).get()["Body"].read()
    # except botocore.exceptions.ClientError as e:


def s3_csv_to_df(bucket_name, s3_filename):
    blah = read_s3_file(bucket_name, s3_filename)
    foo = StringIO(blah.decode("utf-8"))
    return pd.read_csv(foo)

def big_s3_csv_to_df(bucket_name, s3_filename_prefix, suffixes):
    filenames = [s3_filename_prefix + suff
            for suff in suffixes]
    # return filenames
    parts = [read_s3_file(bucket_name, s3_filename) 
            for s3_filename in filenames ]
    blah = functools.reduce(lambda x, y: x+y, parts)
    foo = StringIO(blah.decode("utf-8"))
    return pd.read_csv(foo)